In [1]:
import pandas as pd
import numpy as np

from scipy.stats import norm

import pyomo.environ as pyo

import gurobipy as gp
from gurobipy import GRB

import openpyxl

from icecream import ic


In [2]:
###
### RETRIEVE AND CLEAN DATA
###
file_path = "Railway services-2024.xlsx"

try:
    df = pd.read_excel(io=file_path, engine="openpyxl")
    ic(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")

# Data cleaning/ enhancing

# Removing unneccesary spaces from 'From' and 'To' columns. 
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

# Adding an indicator when the trip is located on line 400
indicator_line_400 = np.where(df['Line'] == 400.0, 1, 0)
df["line_400"] = indicator_line_400


ic| df.head():    Trip      Departure Time        Arrival Time From To  Demand(μ)  Demand(σ)  \
               0     1 1900-01-01 07:00:00 1900-01-01 07:46:00    M  A        327         64   
               1     2 1900-01-01 07:00:00 1900-01-01 07:57:00    H  M        936        308   
               2     3 1900-01-01 07:02:00 1900-01-01 07:22:00    M  B        461        252   
               3     4 1900-01-01 07:04:00 1900-01-01 08:03:00    M  J        428         10   
               4     5 1900-01-01 07:06:00 1900-01-01 08:13:00    M  F        449        124   
               
                  Line  
               0   100  
               1   800  
               2   200  
               3  1000  
               4   600  


In [3]:
### 
### CREATING THE MODEL
###

##
## Initial values:
##
train_type = [
    "OC", 
    "OH"
    ]
cost_train_type = {
    "OC": 260,
    "OH": 210
}
length_train_type = {
    "OC": 100, 
    "OH": 70
    }
capacity_train_type = {
    "OC": 620, 
    "OH": 420
    }

#
# Data manipulated initial values:
#
cross_section = set(
    df['From'].values + df['To'].values
    ) #NOTE: Not used

trips_using_line400 = ((df.loc[df['line_400'] == 1]).index + 1).to_list() # +1 because the indexes start with 0, but the trips at 1. 


###
### Creation of the Model:
###
def create_general_model():
    m = pyo.ConcreteModel()

    ##
    ## CREATION OF SETS
    ##
    m.trips = pyo.Set(
        initialize = df['Trip']
        )

    m.train_type = pyo.Set(
        initialize = train_type
        )

    ##
    ## CREATION OF PARAMETERS
    ##
    m.cost_train_type = pyo.Param(
        m.train_type, 
        initialize = cost_train_type
    )

    m.length_train_type = pyo.Param(
        m.train_type, 
        initialize = length_train_type
        )

    m.capacity_train_type = pyo.Param(
        m.train_type, 
        initialize = capacity_train_type
        )
    
    #
    # Parameters created using functions:
    #
    m.trips_using_line400 = pyo.Param(
        m.trips,  # Assuming model.cross_section contains all possible cross sections
        initialize={trip: 1 if trip in trips_using_line400 else 0 for trip in m.trips}
    )

    m.passengers_per_trip = pyo.Param(
        m.trips, 
        initialize = {trip: df['Demand(μ)'].loc[trip - 1] for trip in m.trips} # -1 because the trips start with trip 1, but the df with index 0. So, loc[200] results in an error, because there are only 199 records. 
    )

    return m


m = create_general_model()


In [4]:
###
### ADDING SPECIFICS FOR MODEL1
###

##
## Variable specific for Model1
##
def specific_variables_Model1(m):
    #
    # Index set for allocation variable model1
    #
    m.index_set_allocation = pyo.Set(
        initialize = m.trips * m.train_type
        )

    #
    # Variable for allocation in Model1
    #
    # the variable represents the number of trains per type allocated to a trip.
    #
    m.allocation_train_numbers = pyo.Var(
        m.index_set_allocation, 
        domain = pyo.NonNegativeIntegers, 
        name = 'train_allocation', 
        doc = 'The number of trains of a certain type allocated to a cross-section'
    )
    return m

##
## Constraints specific for Model1
##
def specific_constraints_Model1(m):
    #
    # Rule for restricting the combined train length. 
    #
    def rule_maximum_length(
            m, 
            trip
            ):
        combined_train_length = sum(
            m.length_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return combined_train_length <= 300 - 100 * m.trips_using_line400[(trip)]
    m.constr_maximum_length = pyo.Constraint(
        m.trips, 
        rule = rule_maximum_length
        )
    
    #
    # Rule for enforcing the ability to accomodate all passengers. 
    #
    def rule_passenger_limit(
            m, 
            trip
            ):
        available_capacity = sum(
            m.capacity_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return available_capacity >= int(m.passengers_per_trip[(trip)])
    m.constr_passenger_limit = pyo.Constraint(
        m.trips, 
        rule = rule_passenger_limit
        )
    
    # FIXME: Should this be with [0.75, 1] or with [1, 1.25] Same for the other Model
    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 1 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            ) <= 1.25 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 1 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            ) <= 1.25 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    return m


##
## Objective rule specific for Model1
##
def specific_objective_Model1(m):
    #
    # Objective rule to minimize the total cost of all trains. 
    #
    def obj_minimize_total_cost(m):
        total_cost = sum(
            m.cost_train_type[(train_type)] * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips
            )
            for train_type in m.train_type
        )
        return total_cost
    m.objective = pyo.Objective(
        rule=obj_minimize_total_cost,
        sense=pyo.minimize
    )
    return m


##
## Add all variables, constraints and objectives to Model1
##
new_model = create_general_model()
model1 = specific_variables_Model1(m=new_model)
model1 = specific_constraints_Model1(m=model1)
model1 = specific_objective_Model1(m=model1)


In [5]:
###
### ADDING SPECIFICS FOR MODEL2
###


#
# Obtain all compositions
#
def get_compositions(bool_all_combinations:bool = True):
    # Define the lengths of each train type
    OH_length = 100
    OC_length = 70

    OH_cap = 420
    OC_cap = 620

    # Define the maximum length of a train
    max_train_length = 300

    trips = df['Trip']

    # Generate all possible combinations of OH and OC trains
    all_combinations = {}
    allowed_combinations_per_trip = {}

    
    for trip in trips.to_list():
        combination_trip = []
        for num_OH in range(max_train_length // OH_length + 1):
            for num_OC in range(max_train_length // OC_length + 1):
            
                length = num_OH * OH_length + num_OC * OC_length
                if length <= max_train_length - 100 * df['line_400'].loc[trip - 1]:
                    combination = []
                    if num_OH > 0:  # Append 'OH' only if num_OH is greater than 0
                        combination.extend(['OH'] * num_OH)
                    if num_OC > 0:  # Append 'OC' only if num_OC is greater than 0
                        combination.extend(['OC'] * num_OC)

                    cap = num_OH * OH_cap + num_OC * OC_cap

                    if length != 0 and length <= max_train_length - 100 * df['line_400'].loc[trip - 1]:
                        if OC_cap * num_OC + OH_cap * num_OH >= df['Demand(μ)'].loc[trip - 1]:
                            all_combinations[str(combination)] = [length, cap, num_OC, num_OH]
                            combination_trip.append(str(combination))
        
        allowed_combinations_per_trip[trip] = combination_trip
        

    if bool_all_combinations:
        return all_combinations
    else:
        return allowed_combinations_per_trip


def get_allowed_comp_per_trip_df():

    allowed_comp = pd.DataFrame(index=df['Trip'].tolist(), columns=get_compositions().keys()).fillna(0)

    compositions_per_trip = get_compositions(False)
    for trip in get_compositions(False).keys():
        
        compos_current_trip = compositions_per_trip.get(trip)

        for comp in compos_current_trip:
            allowed_comp.loc[trip, comp] = 1

    return allowed_comp

##
## Parameter specific for Model2
##
def specific_parameter_Model2(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())

    #
    # Set containing the index for the quantification of the number of train types per composition.
    #
    m.index_quantify_compositions = pyo.Set(
        initialize= m.compositions * m.train_type
    )

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    m.quantify_compositions = pyo.Param(
        m.index_quantify_compositions,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in m.compositions for train_type in m.train_type
        }
    )

    return m


##
## Variable specific for Model2
##
def specific_variables_Model2(m):
    # 
    # Index set for allocation variable model1
    # 
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions)


    # 
    # Variable for allocation in Model1
    # 
    # the binary variable indicates the composition used on a trip.
    # 
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip",
        doc = "Indicates which composition is used on a trip"
    )

    return m


## 
## Constraints specific for Model2
## 
def specific_constraints_Model2(m):
    # NOTE: CORRECT
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip):
        return sum(
            m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    #
    # Rule for allowed compositions
    #
    allowed_df = get_allowed_comp_per_trip_df()
    def rule_allowed_compos(m, trip, composition):
        return m.allocation_compositions[(trip, composition)] <= allowed_df.loc[trip, composition]
    m.constr_allowed_compos = pyo.Constraint(
        m.trips, 
        m.compositions, 
        rule=rule_allowed_compos
    )

    return m


    
def specific_objective_Model2(m):
    m.cost_train_type.pprint()
    def obj_minimize_total_cost(m):
        return m.cost_train_type["OH"] * sum(
                m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            ) + (m.cost_train_type["OC"]) * sum(
                m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            )
    m.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)
    m.objective.pprint()
    return m

new_model = create_general_model()
model2 = specific_parameter_Model2(m=new_model)
model2 = specific_variables_Model2(m=model2)
model2 = specific_constraints_Model2(m=model2)
model2 = specific_objective_Model2(m=model2)

cost_train_type : Size=2, Index=train_type, Domain=Any, Default=None, Mutable=False
    Key : Value
     OC :   260
     OH :   210
objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 210*(0*allocation_compositions[1,"['OC']"] + 0*allocation_compositions[2,"['OC']"] + 0*allocation_compositions[3,"['OC']"] + 0*allocation_compositions[4,"['OC']"] + 0*allocation_compositions[5,"['OC']"] + 0*allocation_compositions[6,"['OC']"] + 0*allocation_compositions[7,"['OC']"] + 0*allocation_compositions[8,"['OC']"] + 0*allocation_compositions[9,"['OC']"] + 0*allocation_compositions[10,"['OC']"] + 0*allocation_compositions[11,"['OC']"] + 0*allocation_compositions[12,"['OC']"] + 0*allocation_compositions[13,"['OC']"] + 0*allocation_compositions[14,"['OC']"] + 0*allocation_compositions[15,"['OC']"] + 0*allocation_compositions[16,"['OC']"] + 0*allocation_compositions[17,"['OC']"] + 0*allocation_compositions[18,"['OC']"] + 0*allocation_comp

In [6]:
###
### ADDING ADDITIONAL REQUIREMENTS FOR MODEL4
###

def get_probs():
    def calc_prob(cap, mu, sigma):
        z_score = (cap - mu) / sigma
        prob = norm.cdf(z_score)
        return prob
    
    compos = get_compositions()
    probs = pd.DataFrame(index=df['Trip'].tolist(), columns=compos.keys())

    for trip in df['Trip'].tolist():
        for comp in compos.keys():
            probs.loc[trip, comp] = calc_prob(
                    cap=compos.get(comp)[1],
                    mu=df['Demand(μ)'].loc[trip - 1],
                    sigma=df['Demand(σ)'].loc[trip - 1]
                )
    
    return probs
probs = get_probs()
    
##
## Parameter specific for Model4
##
def specific_parameter_Model4(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())
    
    #m.days = pyo.Set(initialize = list(range(1, 251)))

    #
    # Set containing the index for the quantification of the number of train types per composition.
    # 
    """m.index_quantify_compositions = pyo.Set(
        initialize= m.compositions * m.train_type * m.days
    )"""

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    m.quantify_compositions = pyo.Param(
        m.compositions * m.train_type,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in m.compositions for train_type in m.train_type
        }
    )
    m.demand_upperbound = 5000

    return m


##
## Variable specific for Model4
##
def specific_variables_Model4(m):
    # 
    # Index set for allocation variable model1
    # 
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions)
    #m.index_set_satisfaction = pyo.Set(initialize = m.trips * m.compositions * m.days)


    # 
    # Variable for allocation in Model3
    # 
    # the binary variable indicates the composition used on a trip.
    # 
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip on a day",
        doc = "Indicates which composition is used on a trip on a specific day"
    )

    return m


## 
## Constraints specific for Model4
## 
def specific_constraints_Model4(m, random_demand: bool = False):
    # NOTE: CORRECT
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip):
        return sum(
            m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1_4(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1_4
    )
    def rule_difference_between_number_of_train_types2_4(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2_4
    )

    #
    # Rule for allowed compositions
    #
    if random_demand:
        allowed_df = get_allowed_comp_per_trip_df(random=True)
    else:
        allowed_df = get_allowed_comp_per_trip_df()
        
    def rule_allowed_compos(m, trip, composition):
        return m.allocation_compositions[(trip, composition)] <= allowed_df.loc[trip, composition]
    m.constr_allowed_compos = pyo.Constraint(
        m.trips, 
        m.compositions, 
        rule=rule_allowed_compos
    )

    # 
    # Capacity constraints
    #
    def rule_125days_satisfied(m, trip):
        return sum(
                probs.loc[trip, composition] * m.allocation_compositions[(trip, composition)] 
                for composition in m.compositions
                ) >= 125/250 #FIXME
    m.constr_125days = pyo.Constraint(
        m.trips, 
        rule=rule_125days_satisfied
    )
    #m.constr_125days.pprint()
    def rule_81_percent_satisfied(m):    
        return sum(
                probs.loc[trip, composition] * m.allocation_compositions[(trip, composition)] 
                for trip in m.trips
                for composition in m.compositions
                ) >= 0.81 * 50000/250
    m.constr_81_persent = pyo.Constraint(
        rule=rule_81_percent_satisfied
    )
    m.constr_81_persent.pprint()
   
    return m


#
# Objective function for minimizing total costs.
#
def specific_objective_Model4(m):
    #m.cost_train_type.pprint()
    def obj_minimize_total_cost(m):
        return m.cost_train_type["OH"] * sum(
                m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            ) + (m.cost_train_type["OC"]) * sum(
                m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            )
    m.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)
    #m.objective.pprint()
     
    return m

new_model = create_general_model()
model4 = specific_parameter_Model4(m=new_model)
model4 = specific_variables_Model4(m=model4)
model4 = specific_constraints_Model4(m=model4)
model4 = specific_objective_Model4(m=model4)

constr_81_persent : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [7]:
#
# Model Q5
#
def get_probs(delta: 1):
    def calc_prob(cap, mu, sigma):
        z_score = (cap - mu) / sigma
        prob = norm.cdf(z_score)
        return prob
    
    compos = get_compositions()
    probs = pd.DataFrame(index=df['Trip'].tolist(), columns=compos.keys())

    for trip in df['Trip'].tolist():
        for comp in compos.keys():
            probs.loc[trip, comp] = calc_prob(
                    cap=compos.get(comp)[1],
                    mu=delta * df['Demand(μ)'].loc[trip - 1],
                    sigma=df['Demand(σ)'].loc[trip - 1]
                )
    
    return probs
decreased_probs = get_probs(delta=0.9)
increased_probs = get_probs(delta=1.15)

ic(increased_probs)
ic(decreased_probs)

##
## Variable specific for Model3
##
def specific_variables_Model5(m):
    # 
    # Index set for allocation variable model1
    # 
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions)
    #m.index_set_satisfaction = pyo.Set(initialize = m.trips * m.compositions * m.days)

    # 
    # Variable for allocation in Model3
    # 
    # the binary variable indicates the composition used on a trip.
    # 
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip on a day",
        doc = "Indicates which composition is used on a trip on a specific day"
    )

    return m

def specific_parameter_Model5(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())

    return m


#
# Constraint
#

def specific_constraints_Model5(m, sol: pd.DataFrame):
    sol = sol.sum(axis=0).transpose()
    ic(sol)
    compos = get_compositions()

    #
    # Constraints restricting the number of traintypes.
    #
    def rule_number_per_type_OC(m, compo):
        return sum(
            compos.get(compo)[2] * m.allocation_compositions[trip, compo] 
                for trip in m.trips
            ) <= sol.loc[compo] * compos.get(compo)[2]
    m.constr_number_per_type_OC = pyo.Constraint(
        m.compositions,
        rule=rule_number_per_type_OC
    )
    def rule_number_per_type_OH(m, compo):
        return sum(
            compos.get(compo)[3] * m.allocation_compositions[trip, compo] 
                for trip in m.trips
            ) <= sol.loc[compo] * compos.get(compo)[3]
    m.constr_number_per_type_OH = pyo.Constraint(
        m.compositions,
        rule=rule_number_per_type_OH
    )

    def rule_one_composition_per_trip(m, trip):
        return sum(
            m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        rule=rule_one_composition_per_trip
    )

    compos = get_compositions()
    def rule_length(m, trip, composition):
        OH_length = 100
        OC_length = 70
        
        length = compos.get(composition)[3] * OH_length  + compos.get(composition)[2] * OC_length

        return length * m.allocation_compositions[(trip, composition)] <= 300 - 100 * df['line_400'].loc[trip - 1]
    m.constr_length = pyo.Constraint(
        m.trips, 
        m.compositions, 
        rule=rule_length
    )
    return m

#
# Objective function for minimizing total costs.
#
def specific_objective_Model5(m):
    days = range(1, 6)
    
    def obj_maximize_prob_satisfy_capacity(m):
        return 50 / 50000 * sum(
            ((day % 2 == 0 or day % 4 == 0) * increased_probs.loc[trip, composition] 
             + (day % 2 != 0 and day % 4 != 0) * decreased_probs.loc[trip, composition]
            ) * m.allocation_compositions[trip, composition]
            for trip in m.trips
            for composition in m.compositions
            for day in days
        )
    m.objective = pyo.Objective(rule=obj_maximize_prob_satisfy_capacity, sense=pyo.maximize)
    m.objective.pprint()
    
    return m


# initialized later because it needs the solution of 4

ic| increased_probs:        ['OC'] ['OC', 'OC'] ['OC', 'OC', 'OC'] ['OC', 'OC', 'OC', 'OC']  \
                     1    0.999931          1.0                1.0                      1.0   
                     2    0.069194     0.702349           0.994523                 0.999997   
                     3    0.639285     0.997575                1.0                      1.0   
                     4         1.0          1.0                1.0                      1.0   
                     5    0.798391          1.0                1.0                      1.0   
                     ..        ...          ...                ...                      ...   
                     196  0.715034       0.9945           0.999997                      1.0   
                     197  0.674678     0.984098           0.999939                      1.0   
                     198  0.708486     0.975637           0.999654                 0.999999   
                     199       0.0          1.0   

In [8]:
#
# Model extension
#

###
### ADDING SPECIFICS FOR MODEL2
###


#
# Obtain all compositions
#
def get_compositions(bool_all_combinations:bool = True):
    # Define the lengths of each train type
    OH_length = 100
    OC_length = 70

    OH_cap = 420
    OC_cap = 620

    # Define the maximum length of a train
    max_train_length = 300

    trips = df['Trip']

    # Generate all possible combinations of OH and OC trains
    all_combinations = {}
    allowed_combinations_per_trip = {}

    
    for trip in trips.to_list():
        combination_trip = []
        for num_OH in range(max_train_length // OH_length + 1):
            for num_OC in range(max_train_length // OC_length + 1):
            
                length = num_OH * OH_length + num_OC * OC_length
                if length <= max_train_length - 100 * df['line_400'].loc[trip - 1]:
                    combination = []
                    if num_OH > 0:  # Append 'OH' only if num_OH is greater than 0
                        combination.extend(['OH'] * num_OH)
                    if num_OC > 0:  # Append 'OC' only if num_OC is greater than 0
                        combination.extend(['OC'] * num_OC)

                    cap = num_OH * OH_cap + num_OC * OC_cap

                    if length != 0 and length <= max_train_length - 100 * df['line_400'].loc[trip - 1]:
                        if OC_cap * num_OC + OH_cap * num_OH >= df['Demand(μ)'].loc[trip - 1]:
                            all_combinations[str(combination)] = [length, cap, num_OC, num_OH]
                            combination_trip.append(str(combination))
        
        allowed_combinations_per_trip[trip] = combination_trip
        

    if bool_all_combinations:
        return all_combinations
    else:
        return allowed_combinations_per_trip


def get_allowed_comp_per_trip_df():

    allowed_comp = pd.DataFrame(index=df['Trip'].tolist(), columns=get_compositions().keys()).fillna(0)

    compositions_per_trip = get_compositions(False)
    for trip in get_compositions(False).keys():
        
        compos_current_trip = compositions_per_trip.get(trip)

        for comp in compos_current_trip:
            allowed_comp.loc[trip, comp] = 1

    return allowed_comp

##
## Parameter specific for ModelEX
##
def specific_parameter_Model_ex(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())

    #
    # Set containing the index for the quantification of the number of train types per composition.
    #
    m.index_quantify_compositions = pyo.Set(
        initialize= m.compositions * m.train_type
    )

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    m.quantify_compositions = pyo.Param(
        m.index_quantify_compositions,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in m.compositions for train_type in m.train_type
        }
    )

    return m


##
## Variable specific for ModelEX
##
def specific_variables_Model_ex(m):
    # 
    # Index set for allocation variable model1
    # 
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions)


    # 
    # Variable for allocation in Model1
    # 
    # the binary variable indicates the composition used on a trip.
    # 
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip",
        doc = "Indicates which composition is used on a trip"
    )

    m.allocation_driver = pyo.Var(
        m.trips,
        domain=pyo.NonNegativeIntegers,
        name = "Specific number of drivers on a trip",
        doc = 'Indicates how many drives are used in a trip'
    )

    return m


## 
## Constraints specific for ModelEX
## 
def specific_constraints_Model_ex(m):
    # NOTE: CORRECT
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip):
        return sum(
            m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    #
    # Rule for allowed compositions
    #
    allowed_df = get_allowed_comp_per_trip_df()
    def rule_allowed_compos(m, trip, composition):
        return m.allocation_compositions[(trip, composition)] <= allowed_df.loc[trip, composition]
    m.constr_allowed_compos = pyo.Constraint(
        m.trips, 
        m.compositions, 
        rule=rule_allowed_compos
    )

    #
    # Rule for total available number of drivers
    #
    def rule_limit_on_available_drivers(m):
        return sum(
            m.allocation_driver[trip] 
            for trip in m.trips
            ) <= 285
    #m.constr_limit_on_available_drivers = pyo.Constraint(
    #    rule=rule_limit_on_available_drivers
    #)

    def rule_number_of_drivers_per_train(m, trip):
        return 3 * sum(
            m.quantify_compositions[(compo, "OH")] * m.allocation_compositions[(trip, compo)] 
            for compo in m.compositions
            ) + 4 * sum(
            m.quantify_compositions[(compo, "OC")] * m.allocation_compositions[(trip, compo)] 
            for compo in m.compositions
            ) <= 7 * m.allocation_driver[trip]
    m.constr_number_of_drivers_per_train = pyo.Constraint(
        m.trips, 
        rule=rule_number_of_drivers_per_train
    )

    return m

    
def specific_objective_Model_ex(m):
    m.cost_train_type.pprint()
    def obj_minimize_total_cost(m):
        return m.cost_train_type["OH"] * sum(
                m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            ) + (m.cost_train_type["OC"]) * sum(
                m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            ) + 36 * sum(m.allocation_driver[trip] for trip in m.trips)
    m.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)
    m.objective.pprint()
    return m

new_model = create_general_model()
model_ex = specific_parameter_Model_ex(m=new_model)
model_ex = specific_variables_Model_ex(m=model_ex)
model_ex = specific_constraints_Model_ex(m=model_ex)
model_ex = specific_objective_Model_ex(m=model_ex)

cost_train_type : Size=2, Index=train_type, Domain=Any, Default=None, Mutable=False
    Key : Value
     OC :   260
     OH :   210
objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 210*(0*allocation_compositions[1,"['OC']"] + 0*allocation_compositions[2,"['OC']"] + 0*allocation_compositions[3,"['OC']"] + 0*allocation_compositions[4,"['OC']"] + 0*allocation_compositions[5,"['OC']"] + 0*allocation_compositions[6,"['OC']"] + 0*allocation_compositions[7,"['OC']"] + 0*allocation_compositions[8,"['OC']"] + 0*allocation_compositions[9,"['OC']"] + 0*allocation_compositions[10,"['OC']"] + 0*allocation_compositions[11,"['OC']"] + 0*allocation_compositions[12,"['OC']"] + 0*allocation_compositions[13,"['OC']"] + 0*allocation_compositions[14,"['OC']"] + 0*allocation_compositions[15,"['OC']"] + 0*allocation_compositions[16,"['OC']"] + 0*allocation_compositions[17,"['OC']"] + 0*allocation_compositions[18,"['OC']"] + 0*allocation_comp

In [9]:
#
# Additional for sensitivity analysis Model3
#
def simulation_sensitivity_model4():
    new_model = create_general_model()
    model_sim = specific_parameter_Model4(m=new_model)
    model_sim = specific_variables_Model4(m=model_sim)
    model_sim = specific_constraints_Model4(m=model_sim, random_demand=True)
    model_sim = specific_objective_Model4(m=model_sim)

    return model_sim


In [10]:
###
### CREATING THE SOLVING STRUCTURE
###

##
## Solve a specific model
##
def solve_model(model_to_solve, time_limit: int = 90, print_info: bool = True):
    ##
    ## Selecting the solver
    ##
    solver = pyo.SolverFactory('cbc') #FIXME: Either 'cbc' or 'glpk': glpk for gurobi (pip install cbcpy or pip install gurobipy)

    #
    # Solve the model
    #
    results = solver.solve(
        model_to_solve, 
        options={'seconds': time_limit})#, 
        #tee=True)

    #
    # Check the solver status
    #
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        print("Solver terminated successfully. Model is feasible.")
    elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
        print("Solver terminated: Model is infeasible.")
    else:
        print("Solver terminated with non-optimal solution.")

    #
    # Output the information from the solver if necessary. 
    #
    if print_info:
        ic(results)
        ic(results.solver.time)
    
    return results

##
## Obtain solution from Model1
##
def get_solution_df_Model1(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for train_type in model.train_type:
            solution_dict[(trip, train_type)] = model.allocation_train_numbers[(trip, train_type)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.train_type)

    for trip in model.trips:
        for train_type in model.train_type:
            solution_DF.loc[trip, train_type] = solution_dict.get((trip, train_type), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
      
        
    return solution_DF


##
## Obtain solution from Model2
##
def get_solution_df_Model2(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            solution_dict[(trip, composition)] = model.allocation_compositions[(trip, composition)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.compositions)

    for trip in model.trips:
        for composition in model.compositions:
            solution_DF.loc[trip, composition] = solution_dict.get((trip, composition), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
    
    return solution_DF


## 
## Obtain solution from Model4
##
def get_solution_df_Model4(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            solution_dict[(trip, composition)] = model.allocation_compositions[(trip, composition)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns= model.compositions)

    for trip in model.trips:
        for composition in model.compositions:
            solution_DF.loc[trip, composition] = solution_dict.get((trip, composition), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
    
    return solution_DF

## 
## Obtain solution from ModelEX
##
def get_solution_df_Model_ex(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}
    solution_driver_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            solution_dict[(trip, composition)] = model.allocation_compositions[(trip, composition)].value
        
        solution_driver_dict[(trip)] = model.allocation_driver[trip].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns= model.compositions)
    solution_driver_DF = pd.Series(index=model.trips)

    for trip in model.trips:
        for composition in model.compositions:
            solution_DF.loc[trip, composition] = solution_dict.get((trip, composition), np.nan)
        
        solution_driver_DF.loc[trip] = solution_driver_dict.get(trip, np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
        ic(solution_driver_DF)
    
    return solution_DF, solution_driver_DF

def get_solution_df_Model_simulation():
    solutions_time = {}
    solutions_objective = {}
    solutions_dfs = {}

    for i in range(0, 100):
        model_sim = simulation_sensitivity_model4()
        results = solve_model(model_to_solve=model_sim, time_limit=3)
        solution4 = get_solution_df_Model4(model=model_sim, display_solution_df=False)

        if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
 
            solutions_time[i] = results.solver.time
            solutions_objective[i] = pyo.value(model_sim.objective)
            solutions_dfs[i] = solution4
            ic(pyo.value(model_sim.objective))

    ic(sum(solutions_time.values())/len(solutions_time.values()))
    ic(sum(solutions_objective.values())/len(solutions_objective.values()))


In [11]:
###
### Solving the actual models
###

#
# Solving Model1
#
solve_model(model_to_solve=model1, time_limit=1)
solution1 = get_solution_df_Model1(model=model1, display_solution_df=True)


#
# Solving Model2
#
solve_model(model_to_solve=model2, time_limit=1)
solution2 = get_solution_df_Model2(model=model2, display_solution_df=True)

# 
# Solving Model3
#
solve_model(model_to_solve=model4, time_limit=300)
solution4 = get_solution_df_Model4(model=model4, display_solution_df=True)

# 
# Solving Model5
#
new_model = create_general_model()
model5 = specific_parameter_Model5(m=new_model)
model5 = specific_variables_Model5(m=model5)
model5 = specific_constraints_Model5(m=model5, sol=solution4)
model5 = specific_objective_Model5(m=model5)

solve_model(model_to_solve=model5, time_limit=10)
solution5 = get_solution_df_Model4(model=model5, display_solution_df=True)


#
# Solve model Extension
#
solve_model(model_to_solve=model_ex, time_limit=10)
solution_ex, solution_ex_driver = get_solution_df_Model_ex(model=model_ex, display_solution_df=True)


#
# Solving Model Sim // Sensitivity Analysis
#
#get_solution_df_Model_simulation()




containing a solution


ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 72917.778, 'Upper bound': 72940.0, 'Number of objectives': 1, 'Number of constraints': 402, 'Number of variables': 400, 'Number of binary variables': 0, 'Number of integer variables': 400, 'Number of nonzeros': 400, 'Sense': 'minimize'}], 'Solver': [{'Status': 'aborted', 'User time': -1.0, 'System time': 1.0, 'Wallclock time': 1.56, 'Termination condition': 'maxTimeLimit', 'Termination message': 'Optimization terminated because the time expended exceeded the value specified in the seconds parameter.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 361, 'Number of created subproblems': 361}, 'Black box': {'Number of iterations': 1854}}, 'Error rc': 0, 'Time': 1.5801212787628174}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 1.5801212787628174
ic| solution_DF:       OC   OH
                 1    0.0  1.0
                 2    1.0  

Solver terminated with non-optimal solution.


ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 72940.0, 'Upper bound': 72940.0, 'Number of objectives': 1, 'Number of constraints': 198, 'Number of variables': 1509, 'Number of binary variables': 2000, 'Number of integer variables': 2000, 'Number of nonzeros': 1509, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.73, 'Wallclock time': 0.89, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 50, 'Number of created subproblems': 50}, 'Black box': {'Number of iterations': 86}}, 'Error rc': 0, 'Time': 0.9070208072662354}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 0.9070208072662354
ic| solution_DF:     ['OC'] ['OC', 'OC'] ['OC', 'OC', 'OC'] ['OC', 'OC', 'OC', 'OC'] ['OH']  \
              

Solver terminated successfully. Model is feasible.


...

ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 0.80569127, 'Upper bound': 0.80569127, 'Number of objectives': 1, 'Number of constraints': 210, 'Number of variables': 1528, 'Number of binary variables': 1928, 'Number of integer variables': 1928, 'Number of nonzeros': 1517, '

Solver terminated successfully. Model is feasible.


Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.04, 'Wallclock time': 0.05, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.07861685752868652}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 0.07861685752868652
ic| solution_DF:     ['OC'] ['OC', 'OC'] ['OC', 'OC', 'OC'] ['OC', 'OC', 'OC', 'OC'] ['OH']  \
                 1      0.0          0.0                0.0                      0.0    1.0   
                 2      0.0          1.0                0.0                      0.0    0.0   
                 3      1.0          0.0                0.0                      0.0    0.0   
            

Solver terminated successfully. Model is feasible.


10 columns]
ic| solution_driver_DF: 1      1.0
                        2      1.0
                        3      1.0
                        4      1.0
                        5      1.0
                              ... 
                        196    1.0
                        197    1.0
                        198    1.0
                        199    1.0
                        200    1.0
                        Length: 200, dtype: float64


In [17]:
#
# CHECK FEASABILITY MODELS
#

# Get number of trains per thing
compos = get_compositions()
number_of_trains = {}
model_sols = [solution1, solution2, solution4, solution5, solution_ex]
for i in range(0, len(model_sols)):
    sol = model_sols[i]
    sol = sol.sum(axis=0).transpose()
    #ic(sol)

    num_OC = 0
    num_OH = 0
    for compo in compos:
        try:
            if i == 0:
                compo_stripped = compo.strip("[]'")
                num_OC = num_OC + sol.loc[compo_stripped] * compos.get(compo)[2]
                num_OH = num_OH + sol.loc[compo_stripped] * compos.get(compo)[3]
            else:
                num_OC = num_OC + sol.loc[compo] * compos.get(compo)[2]
                num_OH = num_OH + sol.loc[compo] * compos.get(compo)[3]
            

            #ic(compo)
            #ic(compos.get(compo)[2])

            #ic(num_OC)
            #ic(num_OH)
        except:
            pass

    number_of_trains[(i + 1, "OC")] = num_OC
    number_of_trains[(i + 1, "OH")] = num_OH

ic(number_of_trains)

ic((solution2 == solution_ex).sum())

ic(solution_ex_driver[solution_ex_driver != 1])
ic(solution_ex_driver.sum())
#
# CHECK FEASABILITY MODEL2
#

"""# Check allocation to specific train types. 
total_composition_sol2 = solution2.sum(axis=0)
total_composition_sol2 = total_composition_sol2[total_composition_sol2 != 0.0]
ic(total_composition_sol2)

total_alloc_per_trip = solution2.sum(axis=1)
total_alloc_per_trip = total_alloc_per_trip[total_alloc_per_trip != 1.0]
ic(total_alloc_per_trip)

ic((111/89 - 1) <= 0.25)


dataframe = pd.DataFrame(index=model4.trips, columns=model4.days)

for trip in model4.trips:
    for day in model4.days:
        dataframe.loc[trip, day] = model4.passengers_per_trip[(trip, day)]


dataframe = dataframe.max(axis=0)

ic(dataframe[(dataframe > 2000)])
ic(df[df['line_400'] == 1])
ic(get_compositions())
"""

#model3.binairy_capacity_satisfied.pprint()

ic| number_of_trains: {(1, 'OC'): 140.0,
                       (1, 'OH'): 174.0,
                       (2, 'OC'): 140.0,
                       (2, 'OH'): 174.0,
                       (3, 'OC'): 174.0,
                       (3, 'OH'): 144.0,
                       (4, 'OC'): 174.0,
                       (4, 'OH'): 144.0,
                       (5, 'OC'): 140.0,
                       (5, 'OH'): 174.0}
ic| (solution2 == solution_ex).sum(): ['OC']                      200
                                      ['OC', 'OC']                199
                                      ['OC', 'OC', 'OC']          200
                                      ['OC', 'OC', 'OC', 'OC']    200
                                      ['OH']                      200
                                      ['OH', 'OC']                188
                                      ['OH', 'OC', 'OC']          200
                                      ['OH', 'OH']                189
                              

"# Check allocation to specific train types. \ntotal_composition_sol2 = solution2.sum(axis=0)\ntotal_composition_sol2 = total_composition_sol2[total_composition_sol2 != 0.0]\nic(total_composition_sol2)\n\ntotal_alloc_per_trip = solution2.sum(axis=1)\ntotal_alloc_per_trip = total_alloc_per_trip[total_alloc_per_trip != 1.0]\nic(total_alloc_per_trip)\n\nic((111/89 - 1) <= 0.25)\n\n\ndataframe = pd.DataFrame(index=model4.trips, columns=model4.days)\n\nfor trip in model4.trips:\n    for day in model4.days:\n        dataframe.loc[trip, day] = model4.passengers_per_trip[(trip, day)]\n\n\ndataframe = dataframe.max(axis=0)\n\nic(dataframe[(dataframe > 2000)])\nic(df[df['line_400'] == 1])\nic(get_compositions())\n"